# 시작

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

# LangSmith를 활용한 RAG 애플리케이션 평가

# Advanced RAG

In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0 GitPython==3.1.43 langchain-chroma==0.1.4 tavily-python==0.5.0

  Using cached langchain_core-0.3.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.25-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.24-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.23-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.22-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain-0.3.20-py3-none-any.whl.metadata (7.7 kB)
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
  Using cached lang

In [ ]:
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)
documents = loader.load()
print(f"로드된 문서 개수: {len(documents)}")

로드된 문서 개수: 29


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''
다음 문맥만을 고려해 질문에 대답하세요.

문맥: """
{context}
"""

질문: {question}
'''
)

model = ChatOpenAI(model="gpt-4o-mini",temperature=0)
retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요를 알려줘")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


'LangChain은 LLM(대형 언어 모델) 기반의 에이전트 및 애플리케이션을 구축하는 가장 쉬운 방법을 제공하는 프레임워크입니다. 사용자는 10줄 이하의 코드로 OpenAI, Anthropic, Google 등 다양한 모델에 연결할 수 있으며, 사전 구축된 에이전트 아키텍처와 모델 통합을 통해 빠르게 시작할 수 있습니다. LangChain은 에이전트와 자율 애플리케이션을 신속하게 구축하고자 하는 개발자에게 추천됩니다.\n\nLangChain의 주요 기능은 다음과 같습니다:\n\n- **실시간 데이터 증강**: 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결할 수 있습니다.\n- **모델 상호 운용성**: 필요에 따라 모델을 쉽게 교체할 수 있습니다.\n- **신속한 프로토타이핑**: 모듈화된 아키텍처를 통해 애플리케이션을 빠르게 구축하고 반복할 수 있습니다.\n- **생산 준비 기능**: 모니터링, 평가 및 디버깅을 지원하여 신뢰할 수 있는 애플리케이션을 배포할 수 있습니다.\n- **활발한 커뮤니티와 생태계**: 다양한 통합, 템플릿 및 커뮤니티 기여 요소를 활용할 수 있습니다.\n- **유연한 추상화 계층**: 필요에 따라 고수준 또는 저수준 구성 요소를 사용할 수 있습니다.\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

In [ ]:
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="검색 쿼리 목록")

query_generation_prompt = ChatPromptTemplate.from_template("""
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반의 에이전트 및 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 서드파티 통합을 연결하여 AI 애플리케이션 개발을 간소화하는 데 도움을 줍니다. LangChain을 사용하면 10줄 이하의 코드로 OpenAI, Anthropic, Google 등 다양한 모델에 연결할 수 있으며, 사전 구축된 에이전트 아키텍처와 모델 통합을 제공하여 빠르게 시작할 수 있습니다.\n\nLangChain은 다음과 같은 기능을 제공합니다:\n- **실시간 데이터 증강**: 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결할 수 있습니다.\n- **모델 상호 운용성**: 필요에 따라 모델을 쉽게 교체할 수 있습니다.\n- **신속한 프로토타이핑**: 모듈화된 아키텍처를 통해 애플리케이션을 빠르게 구축하고 반복할 수 있습니다.\n- **생산 준비 기능**: 모니터링, 평가 및 디버깅을 지원하여 신뢰할 수 있는 애플리케이션을 배포할 수 있습니다.\n- **활발한 커뮤니티와 생태계**: 다양한 통합, 템플릿 및 커뮤니티 기여 구성 요소를 활용할 수 있습니다.\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

In [ ]:
from langchain_core.documents import Document

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 각 문서의 콘텐츠(문자열)와 그 점수의 매핑을 저장하는 딕셔너리 준비
    content_score_mapping = {}

    # 검색 쿼리마다 반복
    for docs in retriever_outputs:
        # 검색 결과의 문서마다 반복
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 처음 등장한 콘텐츠인 경우 점수를 0으로 초기화
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (순위 + k)) 점수를 추가
            content_score_mapping[content] += 1

    # 점수가 큰 순서로 정렬
    ranked = sorted(content_score_mapping.items(),key=lambda x: x[1], reverse=True) # nopa
    return [content for content,_ in ranked]

rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("Langchain의 개요를 알려줘")


'LangChain은 에이전트 및 LLM(대형 언어 모델) 기반 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 서드파티 통합을 연결하여 AI 애플리케이션 개발을 간소화하며, 기술이 발전함에 따라 미래에 대비할 수 있도록 돕습니다.\n\n주요 기능으로는 다음과 같은 것들이 있습니다:\n\n- **실시간 데이터 증강**: 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결할 수 있습니다.\n- **모델 상호 운용성**: 애플리케이션의 필요에 따라 모델을 쉽게 교체할 수 있습니다.\n- **신속한 프로토타이핑**: 모듈화된 아키텍처를 통해 LLM 애플리케이션을 빠르게 구축하고 반복할 수 있습니다.\n- **생산 준비 기능**: 모니터링, 평가 및 디버깅을 지원하여 신뢰할 수 있는 애플리케이션을 배포할 수 있습니다.\n- **활발한 커뮤니티와 생태계**: 다양한 통합, 템플릿 및 커뮤니티 기여 구성 요소를 활용할 수 있습니다.\n- **유연한 추상화 계층**: 필요에 따라 높은 수준의 체인부터 세부적인 제어가 가능한 저수준 구성 요소까지 작업할 수 있습니다.\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

In [ ]:
!pip install langchain-cohere==0.3.0

In [ ]:
from typing import Any
from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int =3 ) -> list[Document]:
  question = inp["question"]
  documents = inp["documents"]

  cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0",top_n=top_n)
  return cohere_reranker.compress_documents(documents=documents, query= question)

rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "documents": retriever,
    }
    | RunnablePassthrough.assign(context=rerank)
    | prompt | model | StrOutputParser()
)

rerank_rag_chain.invoke("Langchain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반의 애플리케이션과 에이전트를 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 서드파티 통합을 연결하여 AI 애플리케이션 개발을 간소화하며, 기술이 발전함에 따라 미래에 대비할 수 있도록 돕습니다.\n\n주요 특징은 다음과 같습니다:\n\n- **실시간 데이터 증강**: 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결할 수 있습니다.\n- **모델 상호 운용성**: 필요에 따라 모델을 교체할 수 있어, 최적의 선택을 위해 실험할 수 있습니다.\n- **신속한 프로토타이핑**: 모듈화된 아키텍처를 통해 LLM 애플리케이션을 빠르게 구축하고 반복할 수 있습니다.\n- **생산 준비 기능**: 모니터링, 평가 및 디버깅을 지원하여 신뢰할 수 있는 애플리케이션을 배포할 수 있습니다.\n- **활발한 커뮤니티와 생태계**: 다양한 통합, 템플릿 및 커뮤니티 기여 구성 요소를 활용할 수 있습니다.\n- **유연한 추상화 계층**: 필요에 따라 높은 수준의 체인부터 세부적인 제어가 가능한 저수준 구성 요소까지 작업할 수 있습니다.\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retrierver = retriever.with_config({"run_name": "langchain_document_retrierver"})
web_retriever = TavilySearchAPIRetriever(k=3).with_config({"run_name": "web_retriever"})

In [ ]:
from enum import Enum

class Route(str, Enum):
  langchain_document = "langchain_document"
  web = "web"

class RouteOutput(BaseModel):
  route: Route

route_prompt = ChatPromptTemplate.from_template("""
질문에 답변하기 위해 적절한 Retriever를 선택하세요.

질문: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

In [ ]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:
  question = inp["question"]
  route = inp["route"]

  if route == Route.langchain_document:
    return langchain_document_retrierver.invoke(question)
  elif route == Route.web:
    return web_retriever.invoke(question)

  raise ValueError(f"Invalid route: {route}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain,
    } | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

In [ ]:
route_rag_chain.invoke("Langchain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반의 에이전트 및 애플리케이션을 구축하기 위한 프레임워크입니다. 개발자들이 다양한 구성 요소와 서드파티 통합을 연결하여 AI 애플리케이션 개발을 간소화할 수 있도록 돕습니다. LangChain은 10줄 이하의 코드로 OpenAI, Anthropic, Google 등과 연결할 수 있는 사전 구축된 에이전트 아키텍처와 모델 통합을 제공합니다.\n\nLangChain을 사용하면 다음과 같은 이점을 누릴 수 있습니다:\n\n- **실시간 데이터 증강**: 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결할 수 있습니다.\n- **모델 상호 운용성**: 필요에 따라 모델을 쉽게 교체할 수 있습니다.\n- **신속한 프로토타이핑**: 모듈화된 아키텍처를 통해 LLM 애플리케이션을 빠르게 구축하고 반복할 수 있습니다.\n- **생산 준비 기능**: 모니터링, 평가 및 디버깅을 위한 내장 지원을 통해 신뢰할 수 있는 애플리케이션을 배포할 수 있습니다.\n- **활발한 커뮤니티와 생태계**: 다양한 통합, 템플릿 및 커뮤니티 기여 요소를 활용할 수 있습니다.\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

In [ ]:
route_rag_chain.invoke("오늘 날씨는 어때?")

'문맥에 따르면, 날씨는 날마다 달라지며 어떤 날에는 햇빛이 화창하게 빛나고, 다른 날에는 비바람이 몰아치기도 한다고 합니다. 따라서 오늘의 날씨는 구체적으로 언급되지 않았지만, 일반적으로 날씨는 다양하게 변할 수 있습니다.'

In [ ]:
!pip install rank-bm25==0.2.2

In [ ]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config({"run_name": "chroma_retriever"})
bm25_retriever = BM25Retriever.from_documents(documents).with_config({"run_name": "bm25_retriever"})

In [ ]:
from langchain_core.runnables import RunnableParallel

hybrid_retriever = RunnableParallel(
    {"bm25_documents": bm25_retriever, "chroma_documents": chroma_retriever}
) | (lambda x: [x["bm25_documents"], x["chroma_documents"]]) | reciprocal_rank_fusion

In [ ]:
hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

In [ ]:
hybrid_rag_chain.invoke("Langchain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반의 애플리케이션과 에이전트를 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 서드파티 통합을 연결하여 AI 애플리케이션 개발을 간소화하며, 기술이 발전함에 따라 미래에 대비할 수 있도록 돕습니다. LangChain을 사용하면 다양한 데이터 소스와 시스템에 LLM을 쉽게 연결하고, 모델 간의 상호 운용성을 제공하며, 신속한 프로토타입 제작이 가능합니다. 또한, 생산 준비가 완료된 기능을 통해 모니터링, 평가 및 디버깅을 지원하며, 활발한 커뮤니티와 생태계를 통해 지속적인 개선과 최신 AI 개발 동향을 반영할 수 있습니다. \n\nLangChain은 다음과 같은 주요 기능을 제공합니다:\n- 실시간 데이터 증강\n- 모델 상호 운용성\n- 신속한 프로토타입 제작\n- 생산 준비 기능\n- 유연한 추상화 계층\n\nLangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 도구를 제공합니다.'

# 기초

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
output = model.invoke("안녕하세요.")
print(output)

 저는 김민지입니다. 만나서 반가워요. 저는 대학교에서 경영학을 전공하고 있어요. 취미는 음악 감상이고, 특히 팝 음악을 좋아해요. 또한 여행을 좋아해서 여러 나라를 다녀보는 것이 꿈이에요. 앞으로 잘 부탁드립니다!


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요! 저는 존이라고 합니다."),
    AIMessage(content="안녕하세요, 존님! 어떤 도움이 필요하신각요?"),
    HumanMessage(content="제 이름을 아시나요?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

네, 존님이라고 말씀하셨습니다! 어떻게 도와드릴까요?


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요!"),
]

for chunk in model.stream(messages):
  print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({"dish": "김치볶음밥"})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 김치볶음밥


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human","{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish":"카레"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        MessagesPlaceholder("chat_history",optional=True),
        ("human","{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="카레는 어떤 음식인가요?"),
            AIMessage("안녕하세요, 존님! 어떻게 도와드릴까요?"),
        ],
        "input":"제 이름은 아시나요?",
    }
)

print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레는 어떤 음식인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 존님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='제 이름은 아시나요?', additional_kwargs={}, response_metadata={})]
